In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import zipfile
matplotlib.style.use('ggplot')

In [ ]:
#zip_ref = zipfile.ZipFile('wap_am_inf.zip', 'r')
zip_ref = zipfile.ZipFile('Outono_EMAp.zip', 'r')

zip_ref.extractall()
zip_ref.close()

In [ ]:
chat = pd.read_table('_chat.txt')
chat.columns = ['msg']
chat.tail()

In [ ]:
lista = []
data = r'\d+/\d+/\d+ \d+:\d+:\d+'
name = r'.*?\:'
msg = r':.*'

msgs = [i for i in chat.msg]

for i in msgs:
    result = re.search(data, i)
    result2 = re.search(name, i[22:])
    result3 = re.search(msg, i[22:])
    if result and result2 and result3:
        lista.append([result.group(),result2.group()[:-1],result3.group()[1:]])

wpp = pd.DataFrame(lista, columns=['data', 'nome', 'msg'])

In [ ]:
wpp.nome = [x.strip() for x in wpp.nome]
wpp.data = pd.to_datetime(wpp.data)
wpp['day'] = [x.day for x in wpp.data]
#wpp['dia_semana'] = [x.strftime('%a') for x in wpp.data]
wpp['dia_semana'] = [x.weekday() for x in wpp.data]
wpp['month'] = [x.month for x in wpp.data]
wpp['year'] = [x.year for x in wpp.data]
wpp['horario'] = [x.strftime('%H:%M') for x in wpp.data]
wpp['caract'] = [len(''.join(x.split())) for x in wpp.msg]
wpp['audio_'] = [('áudio ocultado' in i)*1 for i in wpp.msg]
wpp['imagem_'] = [('imagem ocultada' in i)*1 for i in wpp.msg]
wpp['msg_sem_midia'] = [(not (('áudio ocultado' in i)*1 or ('imagem ocultada' in i)*1))*1 for i in wpp.msg]

In [ ]:
#wpp['nome'] = wpp['nome'].map({'Kkyo': 'Kayo', '+55 21 97152-5505': 'Zidane', '+55 21 98795-7687': 'Felippe', 'Bravo Rodrigo':'Bravo'})
nomes = {'Renato Aranha':'Renato', 'Kkyo': 'Kayo', '\u202a+55\xa021\xa097152‑5505\u202c': 'Zidane', '\u202a+55\xa021\xa098795‑7687\u202c': 'Felippe', 'Bravo Rodrigo':'Bravo'}
wpp.nome.replace(to_replace=nomes, inplace=True)

In [ ]:
#wpp = wpp[wpp.nome!='Amigos de infância!']
wpp = wpp[wpp.nome!='Outono EMAp']

In [ ]:
wpp.tail()

In [ ]:
pd.DataFrame(wpp.groupby(['year','nome'])[['audio_','imagem_','msg_sem_midia']].sum())

In [ ]:
pd.DataFrame(wpp.groupby('year').nome.value_counts())

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 5))
for j,i in enumerate(range(2017,2019)):
    df = pd.DataFrame(wpp[wpp['year']==i].nome.value_counts())
    qtd_msgs = wpp[wpp['year']==i].msg.count()
    df.plot.bar(fontsize=12, ax=axes[j], title=str(i)+': '+str(qtd_msgs)+' mensagens', colormap='plasma', alpha=0.9)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 6), sharex=True)
for j,i in enumerate(range(2017,2019)):
    col = wpp[(wpp.year==i)].horario.value_counts()
    col.index = pd.to_datetime(col.index)
    menor = pd.DataFrame(col.resample('30min').sum())
    menor.index = [x.time().strftime('%H:%M') for x in menor.index]
    qtd_msgs = wpp[wpp['year']==i].msg.count()
    menor.plot.bar(fontsize=10, ax=axes[j], title = str(i)+': '+str(qtd_msgs)+' msgs', colormap='plasma', alpha=0.9)
    plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt

ano = 2018
filtro = wpp[(wpp['year']==ano)].nome.head(30).unique()
nrows = len(filtro)

fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(12, 10))

for i,j in enumerate(filtro):
    col = wpp[(wpp['nome']==j)&(wpp['year']==ano)].horario.value_counts()
    col.index = pd.to_datetime(col.index)
    col = col.sort_index()
    menor = pd.DataFrame(col.resample('30min').sum())
    menor.index = [x.time().strftime('%H:%M') for x in menor.index]
    qtd_msgs = wpp[(wpp['nome']==j)&(wpp['year']==ano)].msg.count()
    menor.plot.bar(fontsize=8, ax=axes[i], title= str(j)+': '+str(qtd_msgs)+' msgs', colormap='plasma', alpha=0.8)
    #axes[i].set_ylabel(j)
    plt.tight_layout()
    plt.xticks(rotation=90)

In [ ]:
wpp.head()

In [ ]:
wpp['msgss'] = 1

new_analysis = pd.DataFrame(wpp.groupby(['nome'])[['caract','msgss']].sum().sort_values('caract',ascending=False)).reset_index()
new_analysis['media_caract_por_msg'] = round(new_analysis.caract/new_analysis.msgss,1)
new_analysis = new_analysis[(new_analysis.nome!='[04/06/2018 13')&(new_analysis.nome!='[18/05/2017 11')]
new_analysis.sort_values('media_caract_por_msg',ascending=False)

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(figsize=(14, 8))

sns.barplot(y='nome', x='media_caract_por_msg', data=a, order=a.nome.unique(), palette='inferno')